# A1111 Model Installer — **SD 1.5**

Downloads models into your persistent data dir:

- **WEBUI_ROOT** (data): `/workspace/a1111`
- Models go under: `/workspace/a1111/models/...`

### What’s covered (SD1.5)
- **Checkpoints** (e.g., v1-5 base or other SD1.5 .safetensors)
- **VAE** files
- **ControlNet (SD1.5)** models (common ones)
- **ADetailer** optional detectors (YOLO/Face)—
  *ADetailer can auto-download at runtime; this cell lets you prefetch if you want.*

### Notes
- Many models on Hugging Face require a **user token** and/or EULA acceptance.
- You can put your token in the cell below (optional). If a file needs auth and you have no token, it will be skipped with a helpful message.
- The downloader is **idempotent**: existing files are skipped.
- After downloads, use the **Restart WebUI** cell at the bottom.

In [ ]:
# --- Configuration (edit these) ---
import os
from pathlib import Path

# Where A1111 stores data (models, embeddings, extensions, etc.)
WEBUI_ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))

# Optional: Hugging Face token (only needed for gated models)
# You can also set it in environment as HF_TOKEN
HF_TOKEN = os.environ.get("HF_TOKEN", "")  # paste token here if desired

# --- Choose which categories to install (True/False) ---
INSTALL_CHECKPOINTS = True
INSTALL_VAES        = True
INSTALL_CONTROLNET  = True
INSTALL_ADETECTORS  = False   # Optional: YOLO/face detectors for ADetailer

print("WEBUI_ROOT:", WEBUI_ROOT)
print("HF_TOKEN set:", bool(HF_TOKEN))

# Model dest folders
CKPT_DIR   = WEBUI_ROOT / "models" / "Stable-diffusion"
VAE_DIR    = WEBUI_ROOT / "models" / "VAE"
CN_DIR     = WEBUI_ROOT / "models" / "ControlNet"
DET_DIR    = WEBUI_ROOT / "models" / "adetailer"

for d in [CKPT_DIR, VAE_DIR, CN_DIR, DET_DIR]:
    d.mkdir(parents=True, exist_ok=True)

CKPT_DIR, VAE_DIR, CN_DIR, DET_DIR

## Model lists (SD1.5)

You can edit these lists. For Hugging Face, you can specify `repo_id` and `filename` (preferred—handles auth if token is set). For direct links, add https URLs to the `direct` lists.

**Important:** Some popular SD1.5 checkpoints (like `v1-5-pruned-emaonly.safetensors`) require license acceptance, so they’ll only download if your token permits.

In [ ]:
# --- Checkpoints (SD1.5) ---
CHECKPOINTS_HF = [
    # Example (requires EULA acceptance via your token):
    # {"repo_id": "runwayml/stable-diffusion-v1-5", "filename": "v1-5-pruned-emaonly.safetensors", "dest": "sd15-v1-5-pruned-emaonly.safetensors"},
]
CHECKPOINTS_DIRECT = [
    # Add any direct .safetensors links here
]

# --- VAEs ---
VAES_HF = [
    # Example: {"repo_id": "stabilityai/sd-vae-ft-mse-original", "filename": "vae-ft-mse-840000-ema-pruned.safetensors", "dest": "sd15-vae-mse.safetensors"},
]
VAES_DIRECT = [
    # Add any direct VAE links
]

# --- ControlNet (SD1.5) common models ---
CONTROLNET_HF = [
    # Some of these are public; if any 403 appears, you may need a token.
    # {"repo_id": "lllyasviel/ControlNet-v1-1", "filename": "control_v11p_sd15_canny.safetensors"},
    # {"repo_id": "lllyasviel/ControlNet-v1-1", "filename": "control_v11p_sd15_depth.safetensors"},
    # {"repo_id": "lllyasviel/ControlNet-v1-1", "filename": "control_v11p_sd15_openpose.safetensors"},
    # {"repo_id": "lllyasviel/ControlNet-v1-1", "filename": "control_v11p_sd15_scribble.safetensors"},
    # {"repo_id": "lllyasviel/ControlNet-v1-1", "filename": "control_v11p_sd15_softedge.safetensors"},
]
CONTROLNET_DIRECT = [
    # Add any direct ControlNet links
]

# --- ADetailer optional detectors (YOLO / face)
# ADetailer can auto-fetch these; use this list only if you want to pre-download.
ADETECTORS_DIRECT = [
    # Examples (placeholders):
    # "https://huggingface.co/ultralytics/YOLOv8/resolve/main/yolov8n.pt",
    # "https://huggingface.co/ai-forever/face-detection/resolve/main/scrfd_10g_bnkps.onnx",
]

print("Lists ready. Edit above to your taste.")

## Downloader
Supports both **Hugging Face** and **direct HTTPS** URLs. Uses `huggingface_hub` when `repo_id`/`filename` are specified; otherwise raw HTTP download.

If a download fails due to permissions (403), the cell prints a helpful hint—usually adding your **HF token** solves it.

In [ ]:
import os, sys, shutil, json, math
import hashlib, tempfile
from pathlib import Path
from urllib.parse import urlparse
from typing import Optional

import requests

try:
    from huggingface_hub import hf_hub_download
    HF_OK = True
except Exception:
    HF_OK = False

def human(n):
    units = ["B","KB","MB","GB","TB"]
    i = 0
    while n >= 1024 and i < len(units)-1:
        n /= 1024.0
        i += 1
    return f"{n:.1f} {units[i]}"

def ensure_dir(d: Path):
    d.mkdir(parents=True, exist_ok=True)
    return d

def download_direct(url: str, dest: Path) -> Optional[Path]:
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest)
        return dest
    try:
        with requests.get(url, stream=True, timeout=30) as r:
            r.raise_for_status()
            size = int(r.headers.get("Content-Length", 0))
            print(f"→ {url} ({human(size) if size else 'unknown size'})")
            tmp = dest.with_suffix(dest.suffix + ".part")
            with open(tmp, "wb") as f:
                for chunk in r.iter_content(chunk_size=1024*1024):
                    if chunk:
                        f.write(chunk)
            tmp.replace(dest)
            print("✔ Saved:", dest)
            return dest
    except requests.HTTPError as e:
        if e.response is not None and e.response.status_code == 403:
            print("✖ 403 Forbidden. If this is a gated HF file, set HF_TOKEN and try again.")
        else:
            print("✖ HTTP error:", e)
    except Exception as e:
        print("✖ Error:", e)
    return None

def download_hf(repo_id: str, filename: str, dest_dir: Path, dest_name: Optional[str] = None) -> Optional[Path]:
    if not HF_OK:
        print("✖ huggingface_hub is not available. Install it in your image or use direct URLs.")
        return None
    dest_dir.mkdir(parents=True, exist_ok=True)
    dest = dest_dir / (dest_name or filename)
    if dest.exists():
        print("✔ Exists:", dest)
        return dest
    try:
        path = hf_hub_download(repo_id=repo_id, filename=filename, token=(HF_TOKEN or None))
        shutil.copy2(path, dest)
        print("✔ HF downloaded:", dest)
        return dest
    except Exception as e:
        msg = str(e)
        if "403" in msg or "Gated" in msg or "Unauthorized" in msg:
            print(f"✖ HF gated/unauthorized for {repo_id}/{filename}. Set HF_TOKEN and accept the repo's terms.")
        else:
            print("✖ HF error:", e)
        return None

def install_list_hf(items, dest_dir: Path):
    for it in items:
        download_hf(it["repo_id"], it["filename"], dest_dir, it.get("dest"))

def install_list_direct(urls, dest_dir: Path):
    for u in urls:
        name = Path(urlparse(u).path).name
        download_direct(u, dest_dir / name)

print("Downloader ready.")

## Run installers
Edit the lists above, then run this cell. Safe to rerun.

In [ ]:
if INSTALL_CHECKPOINTS:
    print("\n=== Checkpoints (SD1.5) ===")
    install_list_hf(CHECKPOINTS_HF, CKPT_DIR)
    install_list_direct(CHECKPOINTS_DIRECT, CKPT_DIR)

if INSTALL_VAES:
    print("\n=== VAEs ===")
    install_list_hf(VAES_HF, VAE_DIR)
    install_list_direct(VAES_DIRECT, VAE_DIR)

if INSTALL_CONTROLNET:
    print("\n=== ControlNet (SD1.5) ===")
    install_list_hf(CONTROLNET_HF, CN_DIR)
    install_list_direct(CONTROLNET_DIRECT, CN_DIR)

if INSTALL_ADETECTORS:
    print("\n=== ADetailer Optional Detectors ===")
    install_list_direct(ADETECTORS_DIRECT, DET_DIR)

print("\nAll done.")

## Restart WebUI
After downloads, restart A1111 so it can discover models if needed.

In [ ]:
import os, time, subprocess, shlex

print("Attempting to restart the A1111 WebUI service via supervisord …")
try:
    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"
    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    if out.returncode == 0 and "a1111:" in (out.stdout or ""):
        print((out.stdout or out.stderr).strip() or "✔ Restarted via supervisorctl")
    else:
        print("Supervisorctl socket unavailable, forcing process restart…")
        subprocess.run("pkill -f 'launch.py'", shell=True)
        time.sleep(5)
        print("✔ WebUI process terminated, Supervisor will auto-restart it")
    print("Wait ~15 seconds, then reload your WebUI tab.")
except Exception as e:
    print("⚠️ Restart attempt failed:", e)
    print("If needed, run this in a Terminal:")
    print("supervisorctl -c /etc/supervisord.conf restart a1111")